<a href="https://colab.research.google.com/github/SauloHenriqueAguiar/backtestBacktrading/blob/main/daytraderotimizador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yahoo_fin


     |████████████████████████████████| 81 kB 2.2 MB/s 
     |████████████████████████████████| 83 kB 2.1 MB/s 
     |████████████████████████████████| 103 kB 19.2 MB/s 
     |████████████████████████████████| 127 kB 23.0 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=d1f6ef37e35e50a7390e5a1bdc8840c94133cc3bebd0da8bf1149a6c05b470c4
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=0e04ae2a6fa8c2e66fec62f9943e88253951693222d5242fbd0afb39830db45c
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6065 sha256=1e87ca2f67337100a5b3c45b96d182ed3466b448bab88851a10241c59d7af749
  Stored in directory: /root/.cache/pip/wheels/73/ad/a4/0dff4a6ef231fc0dfa12ffbac2a36ce

In [2]:
#instala biblioteca yfinance
!pip install yfinance --upgrade --no-cache-dir

     |████████████████████████████████| 6.3 MB 3.0 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.64-py2.py3-none-any.whl size=24109 sha256=cdb3f89082b1cf94d885af0f1d90d3af25501b327e21a9b4d0e44cf393e1557d
  Stored in directory: /tmp/pip-ephem-wheel-cache-f8ullt3g/wheels/86/fe/9b/a4d3d78796b699e37065e5b6c27b75cff448ddb8b24943c288
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [3]:
#instala request
!pip install requests_html

In [4]:
# Instalar Backtrader
!pip install backtrader

     |████████████████████████████████| 410 kB 2.3 MB/s 


In [5]:
!pip install backtrader[plotting]


In [6]:
# Obtenha os pacotes necessários

import datetime  
import backtrader as bt
from IPython.display import display, Image
import yfinance as yf
import backtrader.analyzers as btanalyzers
import backtrader.feeds as btfeeds
import backtrader.strategies as btstrats
import os
import sys
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])


In [23]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

In [24]:
from yahoo_fin.stock_info import get_data
data = get_data("BTC-USD", start_date="05/10/2021", end_date="11/10/2021", index_as_date = True, interval="1d")
data

,open,high,low,close,adjclose,volume,ticker
2021-05-10,58250.871094,59519.355469,54071.457031,55859.796875,55859.796875,71776546298,BTC-USD
2021-05-11,55847.242188,56872.542969,54608.652344,56704.574219,56704.574219,61308396325,BTC-USD
2021-05-12,56714.531250,57939.363281,49150.535156,49150.535156,49150.535156,75215403907,BTC-USD
2021-05-13,49735.433594,51330.843750,46980.019531,49716.191406,49716.191406,96721152926,BTC-USD
2021-05-14,49682.980469,51438.117188,48868.578125,49880.535156,49880.535156,55737497453,BTC-USD
...,...,...,...,...,...,...,...
2021-11-06,61068.875000,61590.683594,60163.781250,61527.480469,61527.480469,29094934221,BTC-USD
2021-11-07,61554.921875,63326.988281,61432.488281,63326.988281,63326.988281,24726754302,BTC-USD
2021-11-08,63344.066406,67673.742188,63344.066406,67566.828125,67566.828125,41125608330,BTC-USD
2021-11-09,67549.734375,68530.335938,66382.062500,66971.828125,66971.828125,42357991721,BTC-USD


In [25]:
data.to_csv('/content/cripto.csv')

In [26]:
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
        ('printlog', False),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('(MA Period %2d) Ending Value %.2f' %
                 (self.params.maperiod, self.broker.getvalue()), doprint=True)

In [27]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

In [28]:
 # Add a strategy
strats = cerebro.optstrategy(
        TestStrategy,
        maperiod=range(10, 31))

In [29]:
# Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
datapath = os.path.join(modpath, '/content/cripto.csv')

In [30]:
# Create a Data Feed
data = bt.feeds.YahooFinanceCSVData(
        dataname=datapath,
              reverse=False)

In [31]:
# Add the Data Feed to Cerebro
cerebro.adddata(data)

In [32]:
# Set our desired cash start
cerebro.broker.setcash(100000.0)
# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00


In [33]:
 # Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=10)


In [34]:
# Set the commission - 0.1% ... divide by 100 to remove the %
cerebro.broker.setcommission(commission=0.1)

In [35]:
 # Run over everything
cerebro.run(maxcpus=1)

2021-11-10, (MA Period 10) Ending Value 100000.00
2021-11-10, (MA Period 11) Ending Value 100000.00
2021-11-10, (MA Period 12) Ending Value 100000.00
2021-11-10, (MA Period 13) Ending Value 100000.00
2021-11-10, (MA Period 14) Ending Value 100000.00
2021-11-10, (MA Period 15) Ending Value 100000.00
2021-11-10, (MA Period 16) Ending Value 100000.00
2021-11-10, (MA Period 17) Ending Value 100000.00
2021-11-10, (MA Period 18) Ending Value 100000.00
2021-11-10, (MA Period 19) Ending Value 100000.00
2021-11-10, (MA Period 20) Ending Value 100000.00
2021-11-10, (MA Period 21) Ending Value 100000.00
2021-11-10, (MA Period 22) Ending Value 100000.00
2021-11-10, (MA Period 23) Ending Value 100000.00
2021-11-10, (MA Period 24) Ending Value 100000.00
2021-11-10, (MA Period 25) Ending Value 100000.00
2021-11-10, (MA Period 26) Ending Value 100000.00
2021-11-10, (MA Period 27) Ending Value 100000.00
2021-11-10, (MA Period 28) Ending Value 100000.00
2021-11-10, (MA Period 29) Ending Value 100000.00


[[<backtrader.cerebro.OptReturn at 0x7f3930d248d0>],

In [40]:
# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Final Portfolio Value: 100000.00
